# Step 2: Single Agent Consultation

이 단계에서는 CrewAI의 가장 기본적인 단위인 **Single Agent(단일 에이전트)**를 만들어 봅니다.
복잡한 도구 없이, LLM의 지식만으로 반려동물의 증상을 분석하고 조언을 제공하는 '종합 수의사' 에이전트를 구현합니다.

In [6]:
# 필요한 라이브러리 설치 (최초 1회 실행)
try:
    import crewai
except ImportError:
    print("Installing crewai and dependencies...")
    !pip install crewai pandas openai

import os
import pandas as pd
from crewai import Agent, Task, Crew, LLM

# 환경 변수 설정 (API 키 하드코딩)
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY_HERE"\nos.environ["OPENAI_MODEL_NAME"] = "gpt-5-mini" # 기본 모델 설정

## 1. LLM 설정
에이전트가 사용할 두뇌(LLM)를 설정합니다. 여기서는 gpt-5-mini를 사용합니다.

In [7]:
# LLM 설정
my_llm = LLM(
    model="gpt-5-mini",
    temperature=1
)

## 2. Agent 정의 (실습)
**`Agent`**는 특정 역할을 수행하는 AI 직원입니다.
- **Role**: 직책 (예: Pet Health Consultant)
- **Goal**: 목표 (예: 증상을 분석하여 조언 제공)
- **Backstory**: 배경 설정 (예: 10년 경력의 수의사...)

In [8]:
pet_consultant = Agent(
    role="종합 반려동물 건강 컨설턴트",
    goal="""
        보호자가 제공한 반려동물의 증상을 분석하고 종합적인 수의학 상담을 제공합니다.
        긴급도를 평가하고, 예상되는 원인을 제시하며, 실질적인 조언을 한국어로 작성합니다.
    """,
    backstory="""
        당신은 따뜻한 마음을 가진 경험 풍부한 수의사입니다.
        걱정하는 반려인들을 위해 복잡한 의학적 문제를 알기 쉬운 말로 설명하는 데 탁월한 능력을 갖추고 있습니다.
        당신은 항상 동물의 안전과 웰빙을 최우선으로 생각합니다.
    """,
    llm=my_llm,
    verbose=True
)

## 3. Task 정의 (실습)
**`Task`**는 에이전트가 수행해야 할 구체적인 작업입니다.
- **Description**: 작업 내용 (구체적으로 작성)
- **Expected Output**: 결과물의 형식

In [9]:
consultation_task = Task(
    description="""
        다음 반려동물 건강 케이스를 분석하세요:
        <case>
        {case_input}
        </case>

        당신의 분석에는 반드시 다음 내용이 포함되어야 하며 모든 내용은 한국어로 작성해야 합니다:
        1. 긴급도 평가 (위급, 응급, 긴급, 비긴급)
        2. 예상되는 원인 (감별 진단)
        3. 보호자를 위한 향후 권장 조치 및 조언
    """,
    expected_output="""
        마크다운 형식으로 작성된 전문적인 수의학 상담 리포트.
        '긴급도', '예상 원인', '권장 조치' 섹션을 포함해야 하며, 모든 내용은 한국어로 작성되어야 합니다.
    """,
    agent=pet_consultant
)

## 4. Crew 실행 (반복)
Agent와 Task를 하나로 묶어 **`Crew`**를 만들고 실행합니다.
Step 1에서 만든 5개의 데이터를 순회하며 모두 실행해 봅니다.

In [10]:
# 1. 데이터 로드
df_processed = pd.read_csv("./data/processed_sample.csv")
print(f"Loaded {len(df_processed)} cases.")

# 2. Crew 생성
single_agent_crew = Crew(
    agents=[pet_consultant],
    tasks=[consultation_task],
    verbose=True
)

# 3. 반복 실행 및 저장
outputs = []
for i, row in df_processed.iterrows():
    print(f"\n--- Processing Case #{i+1} ---")
    case_input = row['formatted_prompt']
    
    result = single_agent_crew.kickoff(inputs={"case_input": case_input})
    outputs.append(result.raw)
    
    # 개별 결과 저장
    with open(f"./data/single_result_{i}.md", "w", encoding="utf-8") as f:
        f.write(result.raw)

print("\nAll cases processed.")

Loaded 5 cases.

--- Processing Case #1 ---


╭─────────────────────────────────────────── 🚀 Crew Execution Started ───────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name:                                                                                                          │
│  crew                                                                                                           │
│  ID:                                                                                                            │
│  7e6f0356-23a2-4e95-a4d1-d039394e585b                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📋 Task Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Started                                                                                                   │
│  Name:                                                                                                          │
│          다음 반려동물 건강 케이스를 분석하세요:                                                                │
│          <case>                                                                                                 │
│          Symptom:                                                                                               │
│  Diarrhea                                                                                                       │
│                                                                                                                 │
│  Pet Information:                                                                                               │
│  Name                                                                                                           │
│  Lucyfur                                                                                                        │
│  Species                                                                                                        │
│  Dog                                                                                                            │
│  Breed                                                                                                          │
│  Portuguese Podengo Pequeno                                                                                     │
│  Age                                                                                                            │
│  3 years                                                                                                        │
│  Gender                                                                                                         │
│  Female                                                                                                         │
│  Status                                                                                                         │
│  Intact                                                                                                         │
│                                                                                                                 │
│  Issue Details:                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│  More Details from Guardian:                                                                                    │
│  She has had bad stinky gas for days. Eating grass and puked.                                                   │
│          </case>                                                                                                │
│                                                                                                                 │
│          당신의 분석에는 반드시 다음 내용이 포함되어야 하며 모든 내용은 한국어로 작성해야 합니다:               │
│          1. 긴급도 평가 (위급, 응급, 긴급, 비긴급)                                                              │
│          2. 예상되는 원인 (감별 진단)                                                                           │
│          3. 보호자를 위한 향후 권장 조치 및 조언                                                                │
│                                                                                                                 │
│  ID: 4dea38b7-8801-49a2-80ab-2c0c

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: 종합 반려동물 건강 컨설턴트                                                                             │
│                                                                                                                 │
│  Task:                                                                                                          │
│          다음 반려동물 건강 케이스를 분석하세요:                                                                │
│          <case>                                                                                                 │
│          Symptom:                                                                                               │
│  Diarrhea                                                                                                       │
│                                                                                                                 │
│  Pet Information:                                                                                               │
│  Name                                                                                                           │
│  Lucyfur                                                                                                        │
│  Species                                                                                                        │
│  Dog                                                                                                            │
│  Breed                                                                                                          │
│  Portuguese Podengo Pequeno                                                                                     │
│  Age                                                                                                            │
│  3 years                                                                                                        │
│  Gender                                                                                                         │
│  Female                                                                                                         │
│  Status                                                                                                         │
│  Intact                                                                                                         │
│                                                                                                                 │
│  Issue Details:                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│  More Details from Guardian:                                                                                    │
│  She has had bad stinky gas for days. Eating grass and puked.                                                   │
│          </case>                                                                                                │
│                                                                                                                 │
│          당신의 분석에는 반드시 다음 내용이 포함되어야 하며 모든 내용은 한국어로 작성해야 합니다:               │
│          1. 긴급도 평가 (위급, 응급, 긴급, 비긴급)                                                              │
│          2. 예상되는 원인 (감별 진단)                                                                           │
│          3. 보호자를 위한 향후 권장 조치 및 조언                                                                │
│                                              

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: 종합 반려동물 건강 컨설턴트                                                                             │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  # 수의학 상담 리포트 — Lucyfur (3세, 암컷, Portuguese Podengo Pequeno)                                         │
│                                                                                                                 │
│  안녕하세요. 걱정이 많으시겠어요. 제공해주신 정보(며칠간 심한 냄새의 방귀, 잔디 섭취 후 구토, 설사)를 바탕으로  │
│  Lucyfur의 상태를 분석해 드립니다.                                                                              │
│                                                                                                                 │
│  ## 긴급도                                                                                                      │
│  응급(빠른 수의학적 평가 권장)                                                                                  │
│  - 현재 상태는 즉시 생명을 위협하는 급성 위기(예: 쇼크, 심한 탈수, 의식 소실)는 아닌 것으로 보이나, 며칠간      │
│  지속되는 악취가 나는 가스와 설사, 잦은 잔디 섭취 및 구토는 추가 악화(탈수, 세균성·기생충성 감염 확산, 위장관   │
│  손상 등)의 위험이 있어 빠른 평가가 필요합니다.                                                                 │
│  - 특히 다음 증상이 있으면 즉시 응급진료:                                                                       │
│    - 설사에 혈액이 섞이거나(검붉거나 선홍색)                                                                    │
│    - 지속적/반복적인 구토(음수 불가)                                                                            │
│    - 식욕 급감 또는 전혀 먹지 않음                                                                              │
│    - 기력 저하(무기력, 쓰러짐), 고열, 복통(배를 만졌을 때 통증/불편)                                            │
│    - 눈에 보이는 탈수(잇몸 건조, 피부 탄력 저하)                                                                │
│  - 위의 증상 중 하나라도 보이면 즉시 병원 방문하십시오. 경미하고 활동적이며 식욕이 있으면서 설사만 있는         │
│  경우에는 가정관리 후 24–48시간 내 호전 여부를 관찰 가능하지만, 호전 없으면 진료 필요.                          │
│                                                                                                                 │
│  ## 예상 원인 (감별 진단)                                                                                       │
│  (가장 흔한 원인부터 제시)                                                                                      │
│  1. 식이성 위장관염(식탐/부적절한 음식 섭취, 급작스런 사료 변경, 이물질 섭취)                                   │
│     - 잔디 섭취 후 구토한 점과 냄새나는 가스는 흔히 식이 부적응이나 위장 자극과 관련.                           │
│  2. 기생충 감염(회충, 편충, Giardia 등)                                                                         │
│     - 특히 설사와 악취나는 가스는 Giardia 같은 기생충과 관련될 수 있음.                                         │
│  3. 세균성 장염(Clostridium spp., Salmonella 등)                                                                │
│     - 심한 악취의 가스와 설사, 때로 구토 동반.                                                                  │
│  4. 음식 부작용 또는 음식불내증 / 급성 식이과민반응                                                             │
│     - 새로운 간식이나 사람 음식 섭취 여부 확인 필요.                                                            │
│  5. 소화불량 또는 흡수장애(예: 췌장기능저하증은 드물지만 지방성 설사와 악취)                                    │
│  6. 이물(부분적 폐색) 또는 위장관 손상                                                                          │
│     - 반복적인 구토나 복통이 있으면 의심.                                                                       │
│  7. 중독(식물, 쓰레기, 독성 물질) — 드물지만 심각.                                                              │
│  8. 염증성 장질환(만성적 경과이면 고려)                                                                         │
│                                                                                                                 │
│ 

...

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: 종합 반려동물 건강 컨설턴트                                                                             │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  # 수의학 상담 리포트 — Chewy (8세, 수컷, 중성화 완료, 혼합견)                                                  │
│                                                                                                                 │
│  안녕하세요. 이사 후 환경 변화로 심리적 스트레스도 받았겠지만, “오늘 설사 발견 / 먹지 않음·무기력·활동저하 /    │
│  배가 처진(팽만 또는 처짐) 상태”라는 증상 조합은 단순 스트레스성 반응일 수도 있지만 내과적                      │
│  문제(탈수·복부질환·감염·대사질환 등)가 숨어 있을 가능성도 있어 신속한 평가가 필요합니다. 따뜻한 마음으로 도움  │
│  드리겠습니다.                                                                                                  │
│                                                                                                                 │
│  ## 긴급도                                                                                                      │
│  긴급 — 가능한 한 빠른 수의학적 평가 권장(24시간 내로 병원 방문)                                                │
│  - 현재까지 호흡곤란, 쓰러짐, 반복적 구토, 혈변 등 심한 응급 증상이 보고되지는 않았으나,                        │
│    - 식욕상실과 전반적 무기력, 배의 이상(팽만·부종·현저한 처짐)은 심각한 기저질환(췌장염, 복수, 장폐색,         │
│  간·심장 질환, 전신감염 등)의 신호일 수 있습니다.                                                               │
│    - 특히 8세 이상 성견은 만성 질환 또는 급성 악화가 올 수 있으므로 지체하지 말고 24시간 이내 수의사 진료를     │
│  권합니다.                                                                                                      │
│  - 즉시 응급실로 가야 하는 상황(다음 중 하나라도 해당하면 즉시 응급 방문):                                      │
│    - 호흡 곤란, 푸르거나 창백한 잇몸, 의식 저하·경련·실신                                                       │
│    - 복부가 급격히 커지거나 만졌을 때 극심한 통증(낑낑거림)                                                     │
│    - 반복적인 구토로 수분 섭취 불가, 혈액 섞인 구토 또는 설사                                                   │
│                                                                                                                 │
│  ## 예상 원인 (감별 진단)                                                                                       │
│  (증상과 연령·환경변화를 고려한 가능성들 — 빈도/심각도 순으로)                                                  │
│  1. 스트레스성 또는 식이성 위장염(급성 식사부적응, 쓰레기·새로운 음식 섭취)                                     │
│  2. 기생충성 설사(회충, 편충, 지아르디아 등) — 이사·환경 변경 후 노출 가능성 증가                               │
│  3. 세균성 장염 또는 장내 미생물 불균형                                                                         │
│  4. 췌장염(특히 지방성 음식 섭취 후) — 식욕부진·무기력·복부 불편 동반 가능                                      │
│  5. 복수(복강 내 액체저류) 또는 복강 종양(간·복막·비장 질환) — “배가 처진/팽만” 느낌이 있는 경우                │
│  6. 장폐색(이물, 장중첩) — 복통·구토·심한 무기력 동반 시 의심                                                   │
│  7. 간·신장 질환 또는 대사성 질환(예: 당뇨, 전해질 불균형)                                                      │
│  8. 심부전(흉수·복수 유발)이나 혈액응고 문제, 패혈증 등 전신성 질환                                             │
│  9. 행동적 우울(환경 변화에 따른) — 의학적 문제 배제 후 고려                                                    │
│                                                                                                                 │
│  정확한 진단을 위해서는 신체검진 및 검사(혈액검사, 소변검사, 대변검사, 복부 방사선/초음파 등)가 필요합니다.     │
│                                                                                                                 │
│  ## 권장 조치 (보호자를 위한 단계별 실무 지침)                                                                  │
│                                                                                                                 │
│  1) 지금 당장 하실 일 (집에서)                                                                                  │
│  

╭────────────────────────────────────────────── 📋 Task Completion ───────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name:                                                                                                          │
│                                                                                                                 │
│          다음 반려동물 건강 케이스를 분석하세요:                                                                │
│          <case>                                                                                                 │
│          Symptom:                                                                                               │
│  Diarrhea                                                                                                       │
│                                                                                                                 │
│  Pet Information:                                                                                               │
│  Name                                                                                                           │
│  Chewy                                                                                                          │
│  Species                                                                                                        │
│  Dog                                                                                                            │
│  Breed                                                                                                          │
│  Mixed                                                                                                          │
│  Age                                                                                                            │
│  8 years                                                                                                        │
│  Gender                                                                                                         │
│  Male                                                                                                           │
│  Status                                                                                                         │
│  Neutered or spayed                                                                                             │
│                                                                                                                 │
│  Issue Details:                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│  More Details from Guardian:                                                                                    │
│  I just noticed it today,but we have moved and this was hard on him because my son has an Aussie doodle and me  │
│  and chewy raised her and trained her and we lived with my son for the last 2 or 4 years He used to eat         │
│  Everytime I feed peeper ,just cause he is stingy.But ever since it's just me and him and we have always been   │
│  alright on our own.Something is different this time he seems so depressed and just not interested in anything  │
│  even eating.He has the saggy belly now and haven't been gone 2 or three weeks.Im really worried.??He's never   │
│  not been by my side please tell me I'm just being overly cautious.not sure if that's the word I'm looking      │
│  for.He was my emotional support animal and if that might help on the sh

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name:                                                                                                          │
│  crew                                                                                                           │
│  ID:                                                                                                            │
│  7e6f0356-23a2-4e95-a4d1-d039394e585b                                                                           │
│  Final Output: # 수의학 상담 리포트 — Chewy (8세, 수컷, 중성화 완료, 혼합견)                                    │
│                                                                                                                 │
│  안녕하세요. 이사 후 환경 변화로 심리적 스트레스도 받았겠지만, “오늘 설사 발견 / 먹지 않음·무기력·활동저하 /    │
│  배가 처진(팽만 또는 처짐) 상태”라는 증상 조합은 단순 스트레스성 반응일 수도 있지만 내과적                      │
│  문제(탈수·복부질환·감염·대사질환 등)가 숨어 있을 가능성도 있어 신속한 평가가 필요합니다. 따뜻한 마음으로 도움  │
│  드리겠습니다.                                                                                                  │
│                                                                                                                 │
│  ## 긴급도                                                                                                      │
│  긴급 — 가능한 한 빠른 수의학적 평가 권장(24시간 내로 병원 방문)                                                │
│  - 현재까지 호흡곤란, 쓰러짐, 반복적 구토, 혈변 등 심한 응급 증상이 보고되지는 않았으나,                        │
│    - 식욕상실과 전반적 무기력, 배의 이상(팽만·부종·현저한 처짐)은 심각한 기저질환(췌장염, 복수, 장폐색,         │
│  간·심장 질환, 전신감염 등)의 신호일 수 있습니다.                                                               │
│    - 특히 8세 이상 성견은 만성 질환 또는 급성 악화가 올 수 있으므로 지체하지 말고 24시간 이내 수의사 진료를     │
│  권합니다.                                                                                                      │
│  - 즉시 응급실로 가야 하는 상황(다음 중 하나라도 해당하면 즉시 응급 방문):                                      │
│    - 호흡 곤란, 푸르거나 창백한 잇몸, 의식 저하·경련·실신                                                       │
│    - 복부가 급격히 커지거나 만졌을 때 극심한 통증(낑낑거림)                                                     │
│    - 반복적인 구토로 수분 섭취 불가, 혈액 섞인 구토 또는 설사                                                   │
│                                                                                                                 │
│  ## 예상 원인 (감별 진단)                                                                                       │
│  (증상과 연령·환경변화를 고려한 가능성들 — 빈도/심각도 순으로)                                                  │
│  1. 스트레스성 또는 식이성 위장염(급성 식사부적응, 쓰레기·새로운 음식 섭취)                                     │
│  2. 기생충성 설사(회충, 편충, 지아르디아 등) — 이사·환경 변경 후 노출 가능성 증가                               │
│  3. 세균성 장염 또는 장내 미생물 불균형                                                                         │
│  4. 췌장염(특히 지방성 음식 섭취 후) — 식욕부진·무기력·복부 불편 동반 가능                                      │
│  5. 복수(복강 내 액체저류) 또는 복강 종양(간·복막·비장 질환) — “배가 처진/팽만” 느낌이 있는 경우                │
│  6. 장폐색(이물, 장중첩) — 복통·구토·심한 무기력 동반 시 의심                                                   │
│  7. 간·신장 질환 또는 대사성 질환(예: 당뇨, 전해질 불균형)                                                      │
│  8. 심부전(흉수·복수 유발)이나 혈액응고 문제, 패혈증 등 전신성 질환                                             │
│  9. 행동적 우울(환경 변화에 따른) — 의학적 문제 배제 후 고려                                                    │
│                                                                                                                 │
│  정확한 진단을 위해서는 신체검진 및 검사(혈액검사, 소변검사, 대변검사, 복부 방사선/초음파 등)가 필요합니다.     │
│                                                                                                                 │
│  ## 권장 조치 (보호자를 위한 단계별 실무 지침)                                                


All cases processed.


╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯